In [ ]:
import os
import requests
from lxml import etree
from selenium import webdriver

search_text = "王祖贤"
start = 0
limit = 15
total = 90

def download(img, title):
dir = "D:\\数据分析\\python test\\query\\" + search_text + "\\"
id = title.replace(u'\u200e', u'').replace(u'?', u'') .replace(u'/', u'or')
if not os.path.exists(dir):
os.makedirs(dir)
try:
pic = requests.get(img, timeout=10)
img_path = dir + str(id) + '.jpg'
fp = open(img_path, 'wb')
fp.write(pic.content)
fp.close()
except requests.exceptions.ConnectionError:
print('图片无法下载')

def crawler_xpath():
src_img = "//div[@class='item-root']/a[@class='cover-link']/img[@class='cover']/@src"
src_title = "//div[@class='item-root']/div[@class='detail']/div[@class='title']/a[@class='title-text']"

for i in range(start,total,limit):
request_url = "https://search.douban.com/movie/subject_search?search_text="+search_text+"&cat=1002&start="+str(i)
driver = webdriver.Chrome()
driver.get(request_url)
html = etree.HTML(driver.page_source)
imgs = html.xpath(src_img)
titles = html.xpath(src_title)
print(imgs,titles)
for img, title in zip(imgs, titles):
download(img, title.text)
if __name__ == '__main__':
crawler_xpath()

In [ ]:
# coding: utf-8
import os

import requests
import json


# 下载图片
def download(src, pic_id, save_path_):
    directory = save_path_ + str(pic_id) + '.jpg'

    try:
        pic = requests.get(src, timeout=10)
        fp = open(directory, 'wb')
        fp.write(pic.content)
        fp.close()
    except requests.exceptions.ConnectionError:
        print('图片如无法下载')


# 获取返回页面内容
def get_resp(query_, limit_, start_):
    url_ = 'https://www.douban.com/j/search_photo?q=' + query_ + '&limit=' + str(limit_) + '&start=' + str(start_)
    html_ = requests.get(url_).text # 得到返回结果
    response_ = json.loads(html_, encoding='utf-8') # 将 JSON 格式转换为 Python 对象
    return response_


query = '王祖贤'
limit = 20
start = 0

#获取图片总数量 '''
total = get_resp(query, limit, start)['total']
print(total)

pic_path = '10' # 相对目录
if not os.path.exists(pic_path):
    os.mkdir(pic_path)
save_path = os.getcwd() + '/' + pic_path + '/'

# 循环 请求全部的 url
for i in range(start, total, limit):
    response = get_resp(query, limit, i)
    for image in response['images']:
        print(image['src']) # 查看当前下载的图片地址
        download(image['src'], image['id'], save_path) # 下载一张图片